In [1]:
import os
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go

init_notebook_mode(connected=True) #do not miss this line

from gensim import corpora, models, similarities

import warnings
warnings.filterwarnings("ignore")

In [2]:
datafile = '/Users/macbook/Desktop/data_elonmusk.csv'

In [3]:
import pandas as pd
tweets = pd.read_csv(datafile, encoding='latin1')
tweets = tweets.assign(Time=pd.to_datetime(tweets.Time)).drop('row ID', axis='columns')

tweets.head(10)

,Tweet,Time,Retweet from,User
0,@MeltingIce Assuming max acceleration of 2 to ...,2017-09-29 17:39:19,NaN,elonmusk
1,RT @SpaceX: BFR is capable of transporting sat...,2017-09-29 10:44:54,SpaceX,elonmusk
2,@bigajm Yup :),2017-09-29 10:39:57,NaN,elonmusk
3,Part 2 https://t.co/8Fvu57muhM,2017-09-29 09:56:12,NaN,elonmusk
4,Fly to most places on Earth in under 30 mins a...,2017-09-29 09:19:21,NaN,elonmusk
5,RT @SpaceX: Supporting the creation of a perma...,2017-09-29 08:57:29,SpaceX,elonmusk
6,BFR will take you anywhere on Earth in less th...,2017-09-29 08:53:00,NaN,elonmusk
7,Mars City\nOpposite of Earth. Dawn and dusk sk...,2017-09-29 06:03:32,NaN,elonmusk
8,Moon Base Alpha https://t.co/voY8qEW9kl,2017-09-29 05:44:55,NaN,elonmusk
9,Will be announcing something really special at...,2017-09-29 02:36:17,NaN,elonmusk


In [4]:
range(len(tweets['Tweet']))

range(0, 3218)

In [5]:
corpus=[]
a=[]
for i in range(len(tweets['Tweet'])):
        a=tweets['Tweet'][i]
        corpus.append(a)

In [17]:
corpus[0:5]

["@MeltingIce Assuming max acceleration of 2 to 3 g's, but in a comfortable direction. Will feel like a mild to moder? https://t.co/fpjmEgrHfC",
 'RT @SpaceX: BFR is capable of transporting satellites to orbit, crew and cargo to the @Space_Station and completing missions to the Moon an?',
 '@bigajm Yup :)',
 'Part 2 https://t.co/8Fvu57muhM',
 'Fly to most places on Earth in under 30 mins and anywhere in under 60. Cost per seat should be? https://t.co/dGYDdGttYd']

In [6]:
import gensim
import logging
import tempfile

TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

from gensim import corpora
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Folder "/var/folders/5r/c9k32sc94rgb48h_n3dmp53c0000gn/T" will be used to save temporary dictionary and corpus.


In [8]:
from nltk.corpus import stopwords
from string import punctuation

# remove common words and tokenize
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

In [9]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'elon.dict'))  # store the dictionary, for future reference
#print(dictionary)

2019-04-30 12:59:24,084 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-04-30 12:59:24,178 : INFO : built Dictionary(11842 unique tokens: ['2', '3', '@meltingice', 'acceleration', 'assuming']...) from 3218 documents (total 30256 corpus positions)
2019-04-30 12:59:24,179 : INFO : saving Dictionary object under /var/folders/5r/c9k32sc94rgb48h_n3dmp53c0000gn/T/elon.dict, separately None
2019-04-30 12:59:24,187 : INFO : saved /var/folders/5r/c9k32sc94rgb48h_n3dmp53c0000gn/T/elon.dict


In [10]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'elon.mm'), corpus)  # store to disk, for later use

2019-04-30 12:59:25,677 : INFO : storing corpus in Matrix Market format to /var/folders/5r/c9k32sc94rgb48h_n3dmp53c0000gn/T/elon.mm
2019-04-30 12:59:25,679 : INFO : saving sparse matrix to /var/folders/5r/c9k32sc94rgb48h_n3dmp53c0000gn/T/elon.mm
2019-04-30 12:59:25,680 : INFO : PROGRESS: saving document #0
2019-04-30 12:59:25,725 : INFO : PROGRESS: saving document #1000
2019-04-30 12:59:25,761 : INFO : PROGRESS: saving document #2000
2019-04-30 12:59:25,801 : INFO : PROGRESS: saving document #3000
2019-04-30 12:59:25,812 : INFO : saved 3218x11842 matrix, density=0.079% (29959/38107556)
2019-04-30 12:59:25,814 : INFO : saving MmCorpus index to /var/folders/5r/c9k32sc94rgb48h_n3dmp53c0000gn/T/elon.mm.index


In [11]:
from gensim import corpora, models, similarities

### Creating a transformation

In [12]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

2019-04-30 12:59:29,096 : INFO : collecting document frequencies
2019-04-30 12:59:29,097 : INFO : PROGRESS: processing document #0
2019-04-30 12:59:29,118 : INFO : calculating IDF weights for 3218 documents and 11841 features (29959 matrix non-zeros)


In [13]:
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors

### LDA:
https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation

In [14]:
total_topics = 2

In [15]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2019-04-30 12:59:37,585 : INFO : using symmetric alpha at 0.5
2019-04-30 12:59:37,587 : INFO : using symmetric eta at 0.5
2019-04-30 12:59:37,591 : INFO : using serial LDA version on this node
2019-04-30 12:59:37,598 : INFO : running online (single-pass) LDA training, 2 topics, 1 passes over the supplied corpus of 3218 documents, updating model once every 2000 documents, evaluating perplexity every 3218 documents, iterating 50x with a convergence threshold of 0.001000
2019-04-30 12:59:37,599 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2019-04-30 12:59:37,601 : INFO : PROGRESS: pass 0, at document #2000/3218
2019-04-30 12:59:40,170 : INFO : merging changes from 2000 documents into a model of 3218 documents
2019-04-30 12:59:40,178 : INFO : topic #0 (0.500): 0.009*"tesla" + 0.007*"model" + 0.004*"@teslamotors:" + 0.004*"like" + 0.003*"first" + 0.003*"falcon" + 0.003*"3" + 0.003*"@teslamotors" + 0.003*

In [16]:
#Show first n important word in the topics:
lda.show_topics(total_topics,5)

[(0,
  '0.011*"tesla" + 0.009*"model" + 0.004*"falcon" + 0.004*"like" + 0.003*"landing"'),
 (1,
  '0.005*"launch" + 0.004*"rocket" + 0.004*"tesla" + 0.003*"@spacex:" + 0.003*"model"')]

In [29]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
1       34.067299 -112.585388       1        1  42.331715
2       28.446142  -56.680176       2        1  21.843781
3       87.052742 -100.641914       3        1  15.476185
0      121.849701  -58.938080       4        1  10.177839
4       74.658195  -28.444054       5        1  10.170484, topic_info=     Category       Freq          Term      Total  loglift  logprob
term                                                               
90    Default   8.000000        rocket   8.000000  30.0000  30.0000
329   Default  12.000000        launch  12.000000  29.0000  29.0000
82    Default   8.000000        falcon   8.000000  28.0000  28.0000
15    Default   7.000000      @spacex:   7.000000  27.0000  27.0000
256   Default   4.000000             9   4.000000  26.0000  26.0000
580   Default  21.000000         model  21.000000  25.0000  25.0000
328   Default   5.000000       landing   5.000000  24.0000  24.0000
272   Default   6.000000           ...   6.000000  23.0000  23.0000
147   Default  10.000000          good  10.000000  22.0000  22.0000
1250  Default   2.000000       station   2.000000  21.0000  21.0000
1347  Default   6.000000  @teslamotors   6.000000  20.0000  20.0000
1929  Default   1.000000           pad   1.000000  19.0000  19.0000
160   Default   5.000000         would   5.000000  18.0000  18.0000
6413  Default   2.000000             v   2.000000  17.0000  17.0000
153   Default   2.000000           yes   2.000000  16.0000  16.0000
881   Default   4.000000         space   4.000000  15.0000  15.0000
1423  Default   1.000000          cape   1.000000  14.0000  14.0000
4248  Default   1.000000     canaveral   1.000000  13.0000  13.0000
1198  Default   3.000000        change   3.000000  12.0000  12.0000
1106  Default   3.000000          live   3.000000  11.0000  11.0000
269   Default   3.000000         stage   3.000000  10.0000  10.0000
170   Default   4.000000       @spacex   4.000000   9.0000   9.0000
1937  Default   2.000000         looks   2.000000   8.0000   8.0000
240   Default   3.000000          need   3.000000   7.0000   7.0000
322   Default   6.000000        dragon   6.000000   6.0000   6.0000
10    Default  12.000000          like  12.000000   5.0000   5.0000
3114  Default   2.000000         super   2.000000   4.0000   4.0000
342   Default   2.000000        coming   2.000000   3.0000   3.0000
1122  Default   3.000000         great   3.000000   2.0000   2.0000
252   Default   2.000000          high   2.000000   1.0000   1.0000
...       ...        ...           ...        ...      ...      ...
328    Topic5   1.404586       landing   5.028376   1.0103  -5.4511
271    Topic5   0.591582         upper   1.309243   1.4913  -6.3158
725    Topic5   0.930384         right   2.833716   1.1719  -5.8630
1347   Topic5   1.488065  @teslamotors   6.473250   0.8155  -5.3934
1038   Topic5   0.476574        thrust   0.950905   1.5949  -6.5320
2271   Topic5   0.604525          fire   1.472838   1.3952  -6.2941
272    Topic5   1.189154           ...   6.207995   0.6331  -5.6176
147    Topic5   1.563674          good  10.914289   0.3426  -5.3438
160    Topic5   1.102155         would   5.510650   0.6763  -5.6936
88     Topic5   0.634262         orbit   1.754892   1.2680  -6.2461
10     Topic5   1.517351          like  12.936421   0.1426  -5.3739
785    Topic5   0.531924     droneship   1.284275   1.4042  -6.4221
696    Topic5   0.707304         still   2.586193   0.9892  -6.1371
3207   Topic5   0.788715        flight   3.615619   0.7631  -6.0282
158    Topic5   0.775626        spacex   3.531858   0.7698  -6.0449
1937   Topic5   0.706169         looks   2.839024   0.8943  -6.1387
580    Topic5   1.482592         model  21.107267  -0.3701  -5.3970
503    Topic5   0.892779          best   5.492943   0.4688  -5.9042
1122   Topic5   0.770221         great   3.688297   0.7194  -6.0519
144    Topic5  